In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1714270793556_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

Import packages

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DecimalType, DoubleType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use "StructType" to make sure every column has correct data type

In [3]:
fileSchema = StructType([StructField('year', IntegerType(),True),
                        StructField('month', StringType(),True),
                        StructField('day', IntegerType(),True),
                        StructField('weekday', StringType(),True),
                        StructField('hour', IntegerType(),True),
                        StructField('atm_status', StringType(),True),
                        StructField('atm_id', StringType(),True),
                        StructField('atm_manufacturer', StringType(),True),
                        StructField('atm_location', StringType(),True),
                        StructField('atm_streetname', StringType(),True),
                        StructField('atm_street_number', IntegerType(),True),
                        StructField('atm_zipcode', IntegerType(),True),
                        StructField('atm_lat', DecimalType(10,3),True),
                        StructField('atm_lon', DecimalType(10,3),True),
                        StructField('currency', StringType(),True),
                        StructField('card_type', StringType(),True),
                        StructField('transaction_amount', IntegerType(),True),
                        StructField('service', StringType(),True),
                        StructField('message_code', StringType(),True),
                        StructField('message_text', StringType(),True),
                        StructField('weather_lat', DoubleType(),True),
                        StructField('weather_lon', DoubleType(),True),
                        StructField('weather_city_id', IntegerType(),True),
                        StructField('weather_city_name', StringType(),True),
                        StructField('temp', DoubleType(),True),
                        StructField('pressure', IntegerType(),True),
                        StructField('humidity', IntegerType(),True),
                        StructField('wind_speed', IntegerType(),True),
                        StructField('wind_deg', IntegerType(),True),
                        StructField('rain_3h', DecimalType(10,3),True),
                        StructField('clouds_all', IntegerType(),True),
                        StructField('weather_id', IntegerType(),True),
                        StructField('weather_main', StringType(),True),
                        StructField('weather_description', StringType(),True),
                        ])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Read data set from HDFS and put to data frame

In [4]:
file1 = spark.read.csv("/user/root/atm/part-m-00000", header = False, schema = fileSchema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Check schema:

In [5]:
file1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: decimal(10,3) (nullable = true)
 |-- atm_lon: decimal(10,3) (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather

In [6]:
file1.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency| card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-

Count to check if we miss data

In [7]:
file1.select().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

Create new column full_data_time combining year, month, day and hour

In [8]:
# Add timestamp
file2 = file1.withColumn('full_date_time', to_timestamp(concat_ws(' ', 'year', 'month', 'day', 'hour'), 'yyyy MMMM d H'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Check data
file2.select('full_date_time', 'year', 'month', 'day', 'hour').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+-------+---+----+
|     full_date_time|year|  month|day|hour|
+-------------------+----+-------+---+----+
|2017-01-01 00:00:00|2017|January|  1|   0|
|2017-01-01 00:00:00|2017|January|  1|   0|
|2017-01-01 00:00:00|2017|January|  1|   0|
|2017-01-01 00:00:00|2017|January|  1|   0|
|2017-01-01 00:00:00|2017|January|  1|   0|
+-------------------+----+-------+---+----+
only showing top 5 rows

Change field name "atm_id" to "atm_number"

In [10]:
# Change atm_id to atm_number
file2 = file2.withColumnRenamed("atm_id", "atm_number")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create temporary table for later use

In [11]:
### creating a temporary table ######
file2.registerTempTable("atm_data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Next, we will create dim and trans tables

# Dim Date

First, create data frame with 'full_date_time', 'year', 'month', 'day', 'hour', 'weekday'

In [12]:
# Dim Date
dim_date1 = file2.select('full_date_time', 'year', 'month', 'day', 'hour', 'weekday').distinct().orderBy('full_date_time', ascending = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Then, create primary key "date_id"

In [13]:
# Create primary key "date_id"
dim_date2 = dim_date1.select(row_number().over(Window.partitionBy().orderBy(dim_date1['full_date_time'])).alias("date_id"), 'full_date_time', 'year', 'month', 'day', 'hour', 'weekday')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Count to check
dim_date2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [15]:
# Create temp table dim date for later use
dim_date2.registerTempTable("dim_date")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Save result to S3
dim_date2.write.parquet("s3://duc-atm-case-study/dim-date/", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Dim Location

Create data frame with 'atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon'

In [17]:
# dim location
dim_location1 = file2.select('atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon').distinct().orderBy('atm_location', ascending = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create primary key "location_id"

In [18]:
# Create primary key "location_id"
dim_location2 = dim_location1.select(row_number().over(Window.partitionBy().orderBy(dim_location1['atm_location'])).alias("location_id"), 'atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Check data
dim_location2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------------+-----------------+-----------+-------+-------+
|location_id|        atm_location|    atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+-----------+--------------------+------------------+-----------------+-----------+-------+-------+
|          1|             Aabybro|      ÃƒËœstergade|                6|       9440| 57.162|  9.730|
|          2|      Aalborg Hallen|      Europa Plads|                4|       9000| 57.044|  9.913|
|          3|Aalborg Storcente...|          Hobrovej|              452|       9200| 57.005|  9.876|
|          4|Aalborg Storcente...|          Hobrovej|              452|       9200| 57.005|  9.876|
|          5|         Aalborg Syd|          Hobrovej|              440|       9200| 57.005|  9.881|
|          6|           AalbÃƒÂ¦k|        Centralvej|                5|       9982| 57.593| 10.412|
|          7|              Aarhus|    SÃƒÂ¸nder Alle|               11|       8000| 56.153| 10.206|


In [20]:
# Change column name
dim_location2 = dim_location2.withColumnRenamed("atm_location", "location")
dim_location2 = dim_location2.withColumnRenamed("atm_streetname", "streetname")
dim_location2 = dim_location2.withColumnRenamed("atm_street_number", "street_number")
dim_location2 = dim_location2.withColumnRenamed("atm_zipcode", "zipcode")
dim_location2 = dim_location2.withColumnRenamed("atm_lat", "lat")
dim_location2 = dim_location2.withColumnRenamed("atm_lon", "lon")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Count to check
dim_location2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [22]:
# Create temp table dim location for later use
dim_location2.registerTempTable("dim_location")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Save result to S3
dim_location2.write.parquet("s3://duc-atm-case-study/dim-location/", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Dim ATM

Create dataframe with 'atm_number', 'atm_manufacturer', 'location_id'

In [24]:
dim_atm1 = spark.sql("select distinct a.atm_number, a.atm_manufacturer, b.location_id from atm_data a, dim_location b \
            where a.atm_location = b.location \
            and a.atm_streetname = b.streetname \
            and a.atm_street_number = b.street_number \
            and a.atm_zipcode = b.zipcode \
            and a.atm_lat = b.lat \
            and a.atm_lon = b.lon \
            order by atm_number")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create primary key "atm_id"

In [25]:
# Create primary key "atm_id"
dim_atm2 = dim_atm1.select(row_number().over(Window.partitionBy().orderBy(dim_atm1['atm_number'])).alias("atm_id"), "atm_number", "atm_manufacturer", "location_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Check data
dim_atm2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+-----------+
|atm_id|atm_number|atm_manufacturer|location_id|
+------+----------+----------------+-----------+
|     1|         1|             NCR|         74|
|     2|        10|             NCR|         76|
|     3|       100|             NCR|         56|
|     4|       101|             NCR|         17|
|     5|       102|             NCR|          3|
|     6|       103| Diebold Nixdorf|        103|
|     7|       104|             NCR|         58|
|     8|       105| Diebold Nixdorf|         76|
|     9|       106|             NCR|         55|
|    10|       107| Diebold Nixdorf|         62|
|    11|       108|             NCR|         47|
|    12|       109| Diebold Nixdorf|          5|
|    13|        11|             NCR|         80|
|    14|       110| Diebold Nixdorf|         41|
|    15|       111| Diebold Nixdorf|          7|
|    16|       112| Diebold Nixdorf|         75|
|    17|       113| Diebold Nixdorf|         89|
|    18|        12| 

In [27]:
dim_atm2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- atm_id: integer (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- location_id: integer (nullable = true)

In [28]:
# Count to check
dim_atm2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [29]:
# Create temp table dim location for later use
dim_atm2.registerTempTable("dim_atm")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Save result to S3
dim_atm2.write.parquet("s3://duc-atm-case-study/dim-atm/", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Dim Card Type

Create dataframe with 'card_type'

In [31]:
# dim card type
dim_card_type1 = file2.select('card_type').distinct().orderBy('card_type', ascending = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create primary key "card_type_id"

In [32]:
# Create primary key "card_type_id"
dim_card_type2 = dim_card_type1.select(row_number().over(Window.partitionBy().orderBy(dim_card_type1['card_type'])).alias("card_type_id"), 'card_type')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Check data
dim_card_type2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|           1|              CIRRUS|
|           2|             Dankort|
|           3|     Dankort - on-us|
|           4|         HÃƒÂ¦vekort|
|           5| HÃƒÂ¦vekort - on-us|
|           6|             Maestro|
|           7|          MasterCard|
|           8|  Mastercard - on-us|
|           9|                VISA|
|          10|        Visa Dankort|
|          11|Visa Dankort - on-us|
|          12|            VisaPlus|
+------------+--------------------+

In [34]:
# Count to check
dim_card_type2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [35]:
# Create temp table dim card type for later use
dim_card_type2.registerTempTable("dim_card_type")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Save result to S3
dim_card_type2.write.parquet("s3://duc-atm-case-study/dim-card-type/", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fact ATM Transaction

Join the primary key of each dim to the original dataframe
- 1) join date_id
- 2) join location_id
- 3) join atm_id
- 4) join card_type_id

### Join dim date

In [37]:
fact_atm1 = spark.sql("select b.date_id, a.* from atm_data a, dim_date b \
                        where a.year = b.year \
                        and a.month = b.month \
                        and a.day = b.day \
                        and a.hour = b.hour \
                        and a.weekday = b.weekday")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# Check schema
fact_atm1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: decimal(10,3) (nullable = true)
 |-- atm_lon: decimal(10,3) (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_cit

In [39]:
# Count to check
fact_atm1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [40]:
# Create temp table for next step
fact_atm1.registerTempTable("fact_atm1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Join dim location

In [41]:
fact_atm2 = spark.sql("select b.location_id, a.* from fact_atm1 a, dim_location b \
                        where a.atm_location = b.location \
                        and a.atm_streetname = b.streetname \
                        and a.atm_street_number = b.street_number \
                        and a.atm_zipcode = b.zipcode \
                        and a.atm_lat = b.lat \
                        and a.atm_lon = b.lon")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# Check schema
fact_atm2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- location_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: decimal(10,3) (nullable = true)
 |-- atm_lon: decimal(10,3) (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lo

In [43]:
# Count to check
fact_atm2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [44]:
# Create temp table for next step
fact_atm2.registerTempTable("fact_atm2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Join dim atm

In [45]:
fact_atm3 = spark.sql("select b.atm_id, a.* from fact_atm2 a, dim_atm b \
                        where a.atm_number = b.atm_number")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# Check schema
fact_atm3.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- atm_id: integer (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: decimal(10,3) (nullable = true)
 |-- atm_lon: decimal(10,3) (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: d

In [47]:
# Count to check
fact_atm3.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [48]:
# Create temp table
fact_atm3.registerTempTable("fact_atm3")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Join dim card type

In [49]:
fact_atm4 = spark.sql("select b.card_type_id, a.* from fact_atm3 a, dim_card_type b \
                        where a.card_type = b.card_type")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# Check schema
fact_atm4.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- card_type_id: integer (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: decimal(10,3) (nullable = true)
 |-- atm_lon: decimal(10,3) (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text:

In [51]:
# Count to check
fact_atm4.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [52]:
# Create temp table for next step
fact_atm4.registerTempTable("fact_atm4")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Final transaction data frame

In [53]:
# select required columns for trans table.
fact_atm5 = spark.sql("select atm_id, \
                              location_id, \
                              date_id, \
                              card_type_id, \
                              atm_status, \
                              currency, \
                              service, \
                              transaction_amount, \
                              message_code, \
                              message_text, \
                              rain_3h, \
                              clouds_all, \
                              weather_id, \
                              weather_main, \
                              weather_description \
                              from fact_atm4")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# change location_id to weather_loc_id
fact_atm5 = fact_atm5.withColumnRenamed("location_id", "weather_loc_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# Count to check
fact_atm5.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [56]:
# Check schema
fact_atm5.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- atm_id: integer (nullable = true)
 |-- weather_loc_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- service: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- rain_3h: decimal(10,3) (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)

In [57]:
# Add primary key trans_id
fact_atm6 = fact_atm5.select(row_number().over(Window.partitionBy().orderBy(fact_atm5['date_id'])).alias("trans_id"), "*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
# Check schema
fact_atm6.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trans_id: integer (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- weather_loc_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- service: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- rain_3h: decimal(10,3) (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)

In [59]:
# Count to check
fact_atm6.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [60]:
# Save result to S3
fact_atm6.write.parquet("s3://duc-atm-case-study/trans-atm/", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…